<a href="https://colab.research.google.com/github/Sylvariane/objets_trouves_sncf/blob/master/Objets_trouv%C3%A9s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyse des objets trouvés dans les trains

Lorsque l'on voyage, on peut être parfois étourdi et cela mène à des pertes d'objets. Je me suis demandée quelles sont les objets les plus oubliés dans le train. Je pense que pour certains l'oubli du portefeuille ou de la veste est chose courante et pour d'autres c'est carrément la valise. La fameuse valise qui constitue un bagage oublié et qui dans certains cas peut conduire à de longs arrêts pour vérifier qu'il n'est pas dangereux. Plongeons dans les données disponibles sur le site : www.data.sncf.com pour en savoir plus. 

## 0. Installation de l'environnement de travail

Avant de commencer notre analyse, il faut débuter par la mise en place de notre environnement de travail. Je commence par me connecter à Google Drive pour pouvoir ouvrir les fichiers dont je vais avoir besoin mais aussi pour exporter ceux que je vais créer. 

In [1]:
# connexion à Google Drive pour importer le fichier
from google.colab import drive
drive.mount('/content/drive')
dir= '/content/drive/My Drive/Formation Data Analyst/Projets personnels/Objets trouvés - SNCF/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Une fois la connexion avec Google Drive effectuée, j'importe les différents modules nécessaires à l'analyse en leur affectant des alias pour que cela soit plus simple de les appeler dans mes commandes Python. 

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Enfin, je crée un premier objet de type DataFrame df qui va contenir le fichier csv que j'ai téléchargé sur le site data.sncf.com.

In [3]:
path = '/content/drive/My Drive/Formation Data Analyst/Projets personnels/Objets trouvés - SNCF/objets-trouves-restitution.csv'
df = pd.read_csv(path, sep=';')

## 1. Nettoyage des données

Toute analyse débute par une partie de nettoyage des données. On affiche les premières lignes du dataFrame pour voir la manière dont il est constitué.

In [4]:
df.head()

,Date,Date et heure de restitution,Gare,Code UIC,Nature d'objets,Type d'objets,Type d'enregistrement
0,2018-01-23T14:33:43+01:00,NaN,Bourges,87576207.0,Autres divers (préciser),Divers,Objet trouvé
1,2014-03-10T09:07:15+01:00,NaN,Toulouse Matabiau,87611004.0,"Sac de voyage, sac de sport, sac à bandoulière","Bagagerie: sacs, valises, cartables",Objet trouvé
2,2014-02-06T08:45:32+01:00,NaN,Toulouse Matabiau,87611004.0,Sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé
3,2014-03-10T09:14:38+01:00,NaN,Paris Est,87113001.0,"Manteau, veste, blazer, parka, blouson, cape","Vêtements, chaussures",Objet trouvé
4,2014-02-06T09:01:24+01:00,2014-02-07T20:55:39+01:00,Paris Est,87113001.0,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé


In [5]:
df.shape

(681307, 7)

On a donc les données en lien avec la date d'enregistrement de l'objet, la date et l'heure de la restitution, la gare, le code UIC associé à la Gare, la nature de l'objet, le type d'objets et le type d'enregistrement.

In [6]:
df.isna().sum()

Date                                 0
Date et heure de restitution    476015
Gare                               295
Code UIC                           295
Nature d'objets                      0
Type d'objets                        0
Type d'enregistrement                0
dtype: int64

On observe qu'il y 476015 données manquantes concernant les dates et heure de restitution des objets mais en réalité, il s'agit d'objet qui n'ont pas été récupéré. Il manque aussi 295 données sur la gare où se trouve l'objet et le code UIC. Le code UIC est lié à la gare, il s'agit du code d'identification de la gare. Le choix de supprimer le code UIC a été fait.

In [7]:
df = df.drop(columns='Code UIC')

Concernant les données où il n'y avait pas de date de restitution, j'ai remplacé la valeur NaN par Non.

In [8]:
df['Date et heure de restitution'] = df.replace(np.nan, 'Non')
df.head()

,Date,Date et heure de restitution,Gare,Nature d'objets,Type d'objets,Type d'enregistrement
0,2018-01-23T14:33:43+01:00,2018-01-23T14:33:43+01:00,Bourges,Autres divers (préciser),Divers,Objet trouvé
1,2014-03-10T09:07:15+01:00,2014-03-10T09:07:15+01:00,Toulouse Matabiau,"Sac de voyage, sac de sport, sac à bandoulière","Bagagerie: sacs, valises, cartables",Objet trouvé
2,2014-02-06T08:45:32+01:00,2014-02-06T08:45:32+01:00,Toulouse Matabiau,Sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé
3,2014-03-10T09:14:38+01:00,2014-03-10T09:14:38+01:00,Paris Est,"Manteau, veste, blazer, parka, blouson, cape","Vêtements, chaussures",Objet trouvé
4,2014-02-06T09:01:24+01:00,2014-02-06T09:01:24+01:00,Paris Est,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé


In [9]:
df.isna().sum()

Date                              0
Date et heure de restitution      0
Gare                            295
Nature d'objets                   0
Type d'objets                     0
Type d'enregistrement             0
dtype: int64

Après l'inspection visuelle des premières lignes, il faut chercher des informations sur les données dupliquées dans notre jeu de données.

In [10]:
duplicated = df.duplicated()
df[duplicated]

,Date,Date et heure de restitution,Gare,Nature d'objets,Type d'objets,Type d'enregistrement
6523,2018-05-16T19:09:01+02:00,2018-05-16T19:09:01+02:00,Paris Montparnasse,"Téléphone portable protégé (étui, coque,…)","Appareils électroniques, informatiques, appare...",Objet trouvé
8604,2018-06-13T13:10:45+02:00,2018-06-13T13:10:45+02:00,Aéroport Charles de Gaulle 2 TGV,Pièce d'identité personnelle et autre,Pièces d'identités et papiers personnels,Objet trouvé
16881,2019-04-14T15:09:23+02:00,2019-04-14T15:09:23+02:00,Aéroport Charles de Gaulle 2 TGV,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé
20737,2018-09-07T19:12:51+02:00,2018-09-07T19:12:51+02:00,Bordeaux Saint-Jean,"Manteau, veste, blazer, parka, blouson, cape","Vêtements, chaussures",Objet trouvé
23152,2018-06-08T12:25:43+02:00,2018-06-08T12:25:43+02:00,Marseille Saint-Charles,Téléphone portable,"Appareils électroniques, informatiques, appare...",Objet trouvé
...,...,...,...,...,...,...
681245,2020-08-13T14:18:08+02:00,2020-08-13T14:18:08+02:00,Bordeaux Saint-Jean,"Porte-monnaie, portefeuille","Porte-monnaie / portefeuille, argent, titres",Objet trouvé
681246,2020-08-06T17:44:09+02:00,2020-08-06T17:44:09+02:00,Lyon Part Dieu,"Sac d'enseigne (plastique, papier, …)","Bagagerie: sacs, valises, cartables",Objet trouvé
681258,2020-07-13T16:44:33+02:00,2020-07-13T16:44:33+02:00,Paris Gare de Lyon,"Sac de voyage, sac de sport, sac à bandoulière","Bagagerie: sacs, valises, cartables",Objet trouvé
681299,2020-08-10T14:50:07+02:00,2020-08-10T14:50:07+02:00,Nantes,Tente,"Articles de sport, loisirs, camping",Objet trouvé


52180 lignes ont été dupliqués dans notre jeu de données. Pour ne pas fausser les futures analyses, on fait donc le choix de supprimer ces lignes. 

In [11]:
df = df[~duplicated]
df.shape

(629127, 6)

Nos données dupliquées ont été supprimés, on passe à la deuxième étape : la modification de l'organisation des données avant de débuter les analyses.

## 2. Data Wrangling

Pour le moment, on ne possède pas de valeurs chiffrées dans notre tableau. On ne dispose que de date/heure et de labels pour les gares et les objets perdus. La première étape va être de modifier notre tableau pour pouvoir faire des calculs.

La première chose que j'ai fait est de changer le type de la valeur date en type datetime et d'ajouter une colonne année pour travailler uniquement sur les années. Il s'agit d'un choix, il est possible de travailler par mois-année par exemple si on s'intéresse à la période de l'année où les gens oublient plus d'affaires.

In [12]:
df['Date'] = pd.to_datetime(df['Date'])
df['Année'] = df['Date'].apply(lambda x: x.strftime("%Y"))
df.head()

,Date,Date et heure de restitution,Gare,Nature d'objets,Type d'objets,Type d'enregistrement,Année
0,2018-01-23 14:33:43+01:00,2018-01-23T14:33:43+01:00,Bourges,Autres divers (préciser),Divers,Objet trouvé,2018
1,2014-03-10 09:07:15+01:00,2014-03-10T09:07:15+01:00,Toulouse Matabiau,"Sac de voyage, sac de sport, sac à bandoulière","Bagagerie: sacs, valises, cartables",Objet trouvé,2014
2,2014-02-06 08:45:32+01:00,2014-02-06T08:45:32+01:00,Toulouse Matabiau,Sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé,2014
3,2014-03-10 09:14:38+01:00,2014-03-10T09:14:38+01:00,Paris Est,"Manteau, veste, blazer, parka, blouson, cape","Vêtements, chaussures",Objet trouvé,2014
4,2014-02-06 09:01:24+01:00,2014-02-06T09:01:24+01:00,Paris Est,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",Objet trouvé,2014


Une fois cette modification sur la date effectuée, on raccourcit notre tableau pour ne garder que les informations qui nous intéresse et y ajouter une valeur de 1 pour pouvoir commencer le dénombrement de nos objets.

In [13]:
df = df[['Année', 'Gare', 'Nature d\'objets','Type d\'objets']]
df['Nombre d\'objets'] = 1
df.head()

,Année,Gare,Nature d'objets,Type d'objets,Nombre d'objets
0,2018,Bourges,Autres divers (préciser),Divers,1
1,2014,Toulouse Matabiau,"Sac de voyage, sac de sport, sac à bandoulière","Bagagerie: sacs, valises, cartables",1
2,2014,Toulouse Matabiau,Sac à dos,"Bagagerie: sacs, valises, cartables",1
3,2014,Paris Est,"Manteau, veste, blazer, parka, blouson, cape","Vêtements, chaussures",1
4,2014,Paris Est,"Valise, sac sur roulettes","Bagagerie: sacs, valises, cartables",1


On commence par faire une modification de notre tableau pour pouvoir obtenir des valeurs pour chaque objet.

In [16]:
df = df.groupby(['Année', 'Gare','Type d\'objets', 'Nature d\'objets']).sum()
df = df.reset_index()
df.head()

,Année,Gare,Type d'objets,Nature d'objets,Nombre d'objets
0,2013,Angers Saint-Laud,"Appareils électroniques, informatiques, appare...","Ordinateur, ordinateur portable, notebook",3
1,2013,Angers Saint-Laud,"Appareils électroniques, informatiques, appare...",Téléphone portable,4
2,2013,Angers Saint-Laud,"Bagagerie: sacs, valises, cartables","Attaché-case, serviette",1
3,2013,Angers Saint-Laud,"Bagagerie: sacs, valises, cartables",Autre bagagerie,1
4,2013,Angers Saint-Laud,"Bagagerie: sacs, valises, cartables",Autre bagagerie (préciser),1


On finit par enregistrer le nouveau fichier crée au format csv.

In [17]:
df.to_csv(f'{dir}objets_trouves.csv')

## 3. Visualisation des données

Pour la partie 'Visualisation des données', un tableau de bord a été crée avec le logiciel Tableau. Voici le lien du Dashboard : https://public.tableau.com/profile/c.cile.guillot#!/vizhome/objets_perdus/Tableaudebord1?publish=yes